# Téléportation d'un état quantique

![title](figures/teleportation.png)

## 1. Création du circuit

**Principe des mesures différées:** On peut toujours placer les mesures en fin de circuit et les remplacer dans le circuit par des opérateurs contrôlés.

**Principe des mesures implicites:** On peut toujours supposer qu'une ligne d'un circuit quantique est terminée par une mesure, sans perte de généralité. 

Voir Nielsen et Chuang pages 186-187.

\begin{equation}
U(\theta, \phi, \lambda) = \begin{pmatrix} 
    \cos(\frac{\theta}{2}) & -e^{i\lambda}\sin(\frac{\theta}{2}) \\
    e^{i\lambda}\sin(\frac{\theta}{2}) & e^{i(\phi+\lambda)}\cos(\frac{\theta}{2})
\end{pmatrix}
\end{equation}

In [ ]:
from qiskit.circuit import QuantumCircuit
from math import pi

circuit = QuantumCircuit(3)

# Construction de l'état à téléporter (ici |1> )
circuit.x(0)
#circuit.u(pi/3,0,0,0)

# Construction de la paire de Bell
circuit.h(1)
circuit.cx(1,2)

# Téléportation partie 1: CNOT et Hadamard
circuit.barrier()
circuit.cx(0,1)
circuit.h(0)

# Téléportation partie 2: contrôle
circuit.cx(1,2)
circuit.cz(0,2)

circuit.measure_all()

#circuit.draw()
circuit.draw(output="mpl", filename="circuit.pdf")

## 2. Choix de la platforme d'exécution

Plusieurs choix possible:
1. Réel processeur quantique
2. Faux processeur quantique (simulateur qui partage des contraintes similaires: bruit et nombre limité de qubits)
3. Simulateur AER

In [ ]:
#from qiskit_ibm_runtime import QiskitRuntimeService
#service = QiskitRuntimeService(channel="ibm_quantum_platform", token="dcac4a17d85117ed2062c7cf206aa031bc64a33355f28ba95d63af80805170909149fd9e3d0103e77f7cb343c050aa09477a4c6c1efdd796500ceb9df09084cc")

#backend = service.least_busy(simulator=False, operational=True)
#backend = service.least_busy(simulator=True, operational=True)

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
backend = FakeManilaV2()

In [ ]:
#from qiskit_aer import AerSimulator
#backend = AerSimulator()

## 3. La "compilation"

In [ ]:
from qiskit.compiler import transpile
circuit_transpiled = transpile(circuit, backend, optimization_level=3)

print("Transpiled for backend =", backend.name)
circuit_transpiled.draw(output="mpl", filename="circuit_transpiled.pdf")

## 4. Exécution

In [ ]:
job = backend.run(circuit_transpiled, shots=1024)

## 5. Analyse des résultats

In [ ]:
result = job.result()

from qiskit.visualization import plot_histogram
plot_histogram(result.get_counts())

## 6. Détails avec Statevector

In [ ]:
from qiskit.quantum_info import Statevector, Operator
from numpy import sqrt, pi, cos, sin

In [ ]:
theta = pi/3
u = Statevector([cos(theta/2), sin(theta/2)])
display(u.draw("latex"))

In [ ]:
from qiskit.circuit import QuantumCircuit

circuit = QuantumCircuit(3)

circuit.cx(0,1)
circuit.h(0)
circuit.cx(1,2)
circuit.cz(0,2)

circuit.draw(output="mpl")

Nous pouvons convertir le circuit en l'opérateur correspondant:

In [ ]:
TpOp = Operator(circuit)
display(TpOp.draw("latex"))

In [ ]:
theta = pi/3
u = Statevector([cos(theta/2), sin(theta/2)])

print("Etat à téléporter:")
display(u.draw("latex"))

bell = Statevector([1/sqrt(2), 0, 0, 1/sqrt(2)])
input_state = u ^ bell

print("Etat initial à trois qubits:")
display(input_state.draw("latex"))

output_state = input_state.evolve(TpOp)

print("Etat final après action de l'opérateur téléportation:")
display(output_state.draw("latex"))

In [ ]:
from qiskit.quantum_info import schmidt_decomposition
print("Décomposition de Schmidt de l'état final entre (qubits 0 et 1) et (qubit 2):")

res = schmidt_decomposition(output_state, [2])
vec1 = res[0][1]
vec2 = res[0][2]

print ("Amplitude = ", res[0][0])
display(vec1.draw("latex"))
display(vec2.draw("latex"))